# Basic Chatbot
### Trimming history based on last k messages
##### Boilerplate code

In [3]:
import langchain
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

google_llm = ChatGoogleGenerativeAI(
    temperature=0,
    model="gemini-2.0-flash", 
    api_key=google_api_key,
    max_tokens=200
)

openai_llm = ChatOpenAI(
    temperature=0, 
    model="gpt-4", 
    api_key=openai_api_key
)

##### Just to know - Manually trimming history

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ChatMessageHistory

prompt_template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant. Answer all the questions very shortly and briefly'),
    MessagesPlaceholder("history"),
    ('human',"{input}")
])

chain = prompt_template | google_llm | StrOutputParser()

conversation = {}

# {sessionid: [], sessionid: []}

def chat_with_bot(payload):
    input = payload['input']
    session_id = payload['session_id']

    if session_id not in conversation:
        conversation[session_id] = ChatMessageHistory()
    
    history = conversation[session_id]

    history.messages = history.messages[-2:]
    
    history.add_user_message(input)
    res = chain.invoke({"input": input, "history": history.messages})
    history.add_ai_message(res)
    print(f"history messages length: {len(history.messages)}")
    
    return res


res = chat_with_bot({"input": "My name is Praveen", "session_id": "user1"})
print(conversation, "\n", res, "\n")

res = chat_with_bot({"input": "What is google?", "session_id": "user1"})
print(conversation, "\n", res, "\n")

res = chat_with_bot({"input": "Where is it located?", "session_id": "user1"})
print(conversation, "\n", res, "\n")

res = chat_with_bot({"input": "What is my name?", "session_id": "user1"})
print(conversation, "\n", res, "\n")

##### Trimming history in RunnableWithMessageHistory

In [ ]:
from collections import deque
from langchain_core.messages import BaseMessage
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory


prompt_template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant. Answer all the questions very shortly and briefly'),
    MessagesPlaceholder("history"),
    ('human',"{input}")
])

chain = prompt_template | google_llm | StrOutputParser()

store = {}

class WindowedChatMessageHistory(BaseChatMessageHistory):
    def __init__(self, k: int = 2):
        self._messages = deque(maxlen=k*2)

    @property
    def messages(self):
        return list(self._messages)

    def add_message(self, message: str) -> None:
        self._messages.append(message)

    def clear(self):
        self._messages.clear()


def get_session_history(session_id):
    if session_id not in store:
         store[session_id] = WindowedChatMessageHistory(k=2)
    return store[session_id]


chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)


res = chain_with_history.invoke(
    {"input": "My name is Praveen."},
    config={"configurable": {"session_id": "user1"}}
)
print(res, "\n")


res = chain_with_history.invoke(
    {"input": "Who is Donald trump?"},
    config={"configurable": {"session_id": "user1"}}
)
print(res, "\n")


res = chain_with_history.invoke(
    {"input": "Who is Putin?"},
    config={"configurable": {"session_id": "user1"}}
)
print(res, "\n")


res = chain_with_history.invoke(
    {"input": "What is my name?"},
    config={"configurable": {"session_id": "user1"}}
)
print(res, "\n")


Okay, Praveen. 

Donald Trump is a former U.S. President. 

Vladimir Putin is the President of Russia. 

Your name is Praveen. 



In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory

# Where we stash session stuff
store = {}

class SummarizedHistory:
    def __init__(self):
        self.summary = "The conversation is empty."
        self.history = ChatMessageHistory()

    def update(self, user_msg, ai_msg):
        # Add new turn to message history
        self.history.add_user_message(user_msg)
        self.history.add_ai_message(ai_msg)

        # Summarize old summary + new turn
        summarize_prompt = ChatPromptTemplate.from_messages([
            ("system", "Summarize the conversation so far briefly."),
            ("human", "Previous summary:\n{summary}\n\nNew exchange:\nUser: {user}\nAI: {ai}\n\nUpdated summary:")
        ])
        chain = summarize_prompt | google_llm
        self.summary = chain.invoke({
            "summary": self.summary,
            "user": user_msg,
            "ai": ai_msg
        }).content

# Create / get history per session
def get_session(session_id):
    if session_id not in store:
        store[session_id] = SummarizedHistory()
    return store[session_id]

# Example use
session = get_session("abc123")

# User says something
user_input = "Hey, I’m Praveen. I like Python."
ai_output = google_llm.invoke(user_input).content
session.update(user_input, ai_output)
print("AI:", ai_output)
print("Summary:", session.summary)

# Next turn
user_input = "Remind me what I just said?"
ai_output = google_llm.invoke(f"Summary so far: {session.summary}\n\nUser: {user_input}").content
session.update(user_input, ai_output)
print("AI:", ai_output)
print("Summary:", session.summary)


AI: Hi Praveen! It's great to meet you. Python is a fantastic language. What do you enjoy doing with Python? Are you working on any interesting projects or learning anything new? I'm always interested to hear what people are doing with it.
Summary: Praveen introduced himself and mentioned he likes Python. The AI responded, welcoming him and asking what he enjoys doing with Python and if he's working on any projects or learning anything new.
AI: You introduced yourself as Praveen and mentioned that you like Python.
Summary: Praveen introduced himself, mentioning he likes Python. The AI welcomed him, asked about his Python interests and projects, and then reminded him of what he had just said when he asked.


In [ ]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    history = store[session_id]
    # Keep only last 10 messages
    if len(history.messages) > 10:
        history.messages = history.messages[-10:]
    return history



# Summarize old messages when history gets too long
if len(history.messages) > 20:
    old_messages = history.messages[:-10]
    summary = llm.invoke(f"Summarize: {old_messages}").content
    history.clear()
    history.add_message(SystemMessage(content=f"Previous conversation summary: {summary}"))
